In [ ]:
import spatialdata
import sopa.segmentation
import sopa.io
from multiprocessing import Pool

In [ ]:
Load in the data

In [ ]:
path = "/home/fotakis/myScratch/MACSima/data/test_run/new_test/"
sdata = sopa.io.macsima(path)

In [ ]:
sdata

In [ ]:
sdata = sopa.io.macsima(path)

In [ ]:
image_key = "new_test"

In [ ]:
patches = sopa.segmentation.Patches2D(sdata, image_key, patch_width=3000, patch_overlap=40)
patches.write();